# Bozza Random forest

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
import os


heade = ['ID', 'M1', 'M2', 'S1', 'S2', 't_pair', 't_elapsed', 'Mrem', 'Srem', 'esca_v', 'Mtot', 'gen']

path_data = './data/'
dyn = '/Dyn/' # Direi che così è facilmente customizzabile per come ogni autistic* si è sistemato i files.

def extractor(path_data, cluster):
    dataset = {}
    wichcluster = os.listdir(path_data)
    if cluster == 'gc':
        cluster = wichcluster[2]
    if cluster == 'nsc':
        cluster = wichcluster[1]
    if cluster == 'ysc':
        cluster = wichcluster[0]
    for i in os.listdir(path_data + cluster + dyn):
        name = path_data + cluster + '/Dyn/' + str(i) + '/nth_generation.txt'
        dataset[str(i)]=pd.read_csv(name, delimiter=' ', skiprows=1, header=None)#, names=heade)
        dataset[str(i)].drop(dataset[str(i)].columns[[5, 6, 7, 8, 10, 11, 12, 14, 18, 19, 20, 21, 22, 23, 24, 26]], axis=1, inplace=True)
        dataset[str(i)].columns = heade
    return dataset



In [10]:
df_gc = extractor(path_data,'gc') # globular clusters
df_nsc = extractor(path_data,'nsc') # nuclear clusters
df_ysc = extractor(path_data,'ysc') # young star clusters
